Rebuild street graph
====================
Loads a street graph and reallocates roads space, e.g., into a network of one-way streets

In [ ]:
import snman
from snman import osmnx_customized as oxc
from snman.constants import *
import geopandas as gpd

PERIMETER = '_debug'
# Set SAVE_TO_DEBUG = True for saving the results into the _debug folder
# which is automatically used in the QGIS files
SAVE_TO_DEBUG = True

# Set these paths according to your own setup
data_directory = 'C:/Users/lballo/polybox/Research/SNMan/SNMan Shared/data/'
inputs_path = data_directory + 'inputs/'
process_path = data_directory + 'process/' + PERIMETER + '/'

if SAVE_TO_DEBUG:
    export_path = data_directory + 'outputs/' + '_debug' + '/'
else:
    export_path = data_directory + 'outputs/' + PERIMETER + '/'

In [ ]:
# =====================================================================================
# LOAD DATA
# =====================================================================================

print('Load street graph')
G = snman.io.load_street_graph(process_path + 'edges_all_attributes.gpkg', process_path + 'nodes_all_attributes.gpkg')

print('Load rebuilding regions')
# Polygons that define which streets will be reorganized
rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
    inputs_path + 'rebuilding_regions/rebuilding_regions.gpkg'
)

print('Load measurement regions')
# Polygons that define areas where network measures will be calculated
measurement_regions_gdf = snman.io.load_measurement_regions(
    inputs_path + 'measurement_regions/measurement_regions.gpkg'
)

print('Load perimeters')
perimeters_gdf = snman.load_perimeters(inputs_path + 'perimeters/perimeters.shp')

In [ ]:
print('Load POIs')
poi_gpd = snman.io.load_poi(inputs_path + 'poi/poi.gpkg').clip(perimeters_gdf.loc['zrh_north-west'].geometry)

In [ ]:
# =====================================================================================
# GIVEN LANES
# =====================================================================================

if 1:
    print('Set given lanes')
    snman.set_given_lanes(G, bidirectional_for_dead_ends=False)

if 1:
    print('Create directed graph of given lanes')
    G_minimal_graph_input = snman.create_given_lanes_graph(G, hierarchies_to_remove={snman.hierarchy.HIGHWAY})

In [ ]:
# =====================================================================================
# REBUILD
# =====================================================================================

if 1:
    print('Rebuild regions')
    snman.owtop.rebuild_regions(G, rebuilding_regions_gdf, verbose=True)
    snman.generate_lane_stats(G, lanes_attribute=snman.constants.KEY_LANES_DESCRIPTION_AFTER)

In [ ]:
import networkx as nx

H = oxc.truncate.truncate_graph_polygon(
    G,
    measurement_regions_gdf.loc[0].geometry,
    quadrat_width=100,
    retain_all=True
)


In [ ]:
L = snman.graph_tools.street_graph_to_lane_graph(H, MODE_PRIVATE_CARS, KEY_LANES_DESCRIPTION_AFTER)
oxc.plot_graph(L)

In [ ]:
import pandas as pd

# calculate lane graph stats for different modes before and after the rebuild

res = pd.DataFrame([
    snman.graph_tools.calculate_lane_graph_stats(
        snman.graph_tools.street_graph_to_lane_graph(H, MODE_PRIVATE_CARS, KEY_LANES_DESCRIPTION)
    ),
    snman.graph_tools.calculate_lane_graph_stats(
        snman.graph_tools.street_graph_to_lane_graph(H, MODE_PRIVATE_CARS, KEY_LANES_DESCRIPTION_AFTER)
    ),
    snman.graph_tools.calculate_lane_graph_stats(
        snman.graph_tools.street_graph_to_lane_graph(H, MODE_CYCLING, KEY_LANES_DESCRIPTION)
    ),
    snman.graph_tools.calculate_lane_graph_stats(
        snman.graph_tools.street_graph_to_lane_graph(H, MODE_CYCLING, KEY_LANES_DESCRIPTION_AFTER)
    ),
    snman.graph_tools.calculate_lane_graph_stats(
        snman.graph_tools.street_graph_to_lane_graph(H, MODE_TRANSIT, KEY_LANES_DESCRIPTION)
    ),
    snman.graph_tools.calculate_lane_graph_stats(
        snman.graph_tools.street_graph_to_lane_graph(H, MODE_TRANSIT, KEY_LANES_DESCRIPTION_AFTER)
    )
])

res['mode'] = [MODE_PRIVATE_CARS, MODE_PRIVATE_CARS, MODE_CYCLING, MODE_CYCLING, MODE_TRANSIT, MODE_TRANSIT]
res['situation'] = ['before', 'after', 'before', 'after', 'before', 'after']

res = res.set_index(['mode', 'situation']).transpose()

res

In [ ]:
# =====================================================================================
# EXPORT
# =====================================================================================

if 1:
    print('Export network without lanes')
    snman.export_street_graph(G, export_path + 'edges_all_attributes.gpkg', export_path + 'nodes_all_attributes.gpkg')
    snman.export_street_graph(G, export_path + 'edges.gpkg', export_path + 'nodes.gpkg',
        edge_columns=snman.constants.EXPORT_EDGE_COLUMNS
    )

if 0:
    print('Export OSM XML')
    snman.export_osm_xml(G, export_path + 'new_network.osm',{
        'highway', 'lanes', 'lanes:forward', 'lanes:backward', 'lanes:both_ways',
        'cycleway', 'cycleway:lane', 'cycleway:left', 'cycleway:left:lane', 'cycleway:right', 'cycleway:right:lane',
        'bus:lanes:backward', 'bus:lanes:forward', 'vehicle:lanes:backward', 'vehicle:lanes:forward',
        'maxspeed', 'oneway',
        '_connected_component'
    }, uv_tags=True, tag_all_nodes=True)

if 1:
    print('Export network with lanes')
    snman.export_street_graph_with_lanes(G, 'ln_desc', export_path + 'edges_lanes.shp', scaling=4)
    snman.export_street_graph_with_lanes(G, 'ln_desc_after', export_path + 'edges_lanes_after.shp', scaling=4)

In [ ]:
L = snman.graph_tools.street_graph_to_lane_graph(G, MODE_CYCLING, KEY_LANES_DESCRIPTION_AFTER)

# Snap POIs
nodes = oxc.graph_to_gdfs(L, edges=False)
nodes['node_geom'] = nodes.geometry
snapped_poi_gpd = gpd.sjoin_nearest(poi_gpd, nodes, how='inner', max_distance=1000)[['index_right','node_geom']]\
    .rename(columns={'node_geom':'geometry'}).set_geometry('geometry').set_index('index_right')


# Cost List

import itertools as it
import shapely as shp
import pandas as pd

ods = it.permutations(snapped_poi_gpd.index, 2)
a = gpd.GeoDataFrame(ods, columns=['origin', 'destination'])

a['origin_geom'] = snapped_poi_gpd.merge(a, left_index=True, right_on='origin')['geometry']
a['destination_geom'] = snapped_poi_gpd.merge(a, left_index=True, right_on='destination')['geometry']
a['od_line_geom'] = a.apply(
    lambda row: shp.LineString([row['origin_geom'], row['destination_geom']]),
    axis=1).set_crs(2056)
a['cost'] = a.apply(
    lambda row: nx.shortest_path_length(L, source=row['origin'], target=row['destination'], weight='cost'),
    axis=1)
a['od_path_geom'] = a.apply(
    lambda row: shp.LineString(
        nodes.loc[nx.shortest_path(L, source=row['origin'], target=row['destination'], weight='cost')]['geometry'])
    ,axis=1).set_crs(2056)

a_complete = a
a = a[['origin', 'destination', 'cost', 'od_path_geom']]
a = a.set_geometry('od_path_geom')
a.to_file(export_path + 'od.gpkg')

# List of Points
snapped_poi_gpd.to_file(export_path + 'cartogram_points.gpkg')

# OD Matrix
M = pd.DataFrame(index=snapped_poi_gpd.index, columns=snapped_poi_gpd.index)
M.index.name = None

for idx, row in a.iterrows():
    M.loc[row['origin'], row['destination']] = row['cost']
    M.loc[row['origin'], row['origin']] = 0
    M.loc[row['destination'], row['destination']] = 0

M.to_csv(export_path + 'od.csv')